<a href="https://colab.research.google.com/github/Masoud-Ghasemian/BERT/blob/master/chatbot1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import torch
import random
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import nltk
import numpy as np
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
def tokenize(sentence):
  return nltk.word_tokenize(sentence)

In [12]:
sentence = 'hey how are you'
tokenize(sentence)

['hey', 'how', 'are', 'you']

In [13]:
def stem(word):
  return stemmer.stem(word.lower())

In [14]:
words = ['dogs', 'Cat', 'talked']
stemmed_words = [stem(word) for word in words]
print(stemmed_words)

['dog', 'cat', 'talk']


In [15]:
def bag_of_words(tokenized_sentence, all_words):
  bag = np.zeros(len(all_words), dtype=np.float32)
  tokenized_sentence = [stem(w) for w in tokenized_sentence]
  for idx, w in enumerate(all_words):
    if w in tokenized_sentence:
      bag[idx] = 1.0

  return bag

In [16]:
import json
with open('intents.json', 'r') as f:
  intents = json.load(f)

all_words = []
tags = []
xy = []
for intent in intents['intents']:
  tag = intent['tag']
  tags.append(tag)
  for pattern in intent['patterns']:
    w = tokenize(pattern)
    all_words.extend(w)
    xy.append((w, tag))
ignore_words = ['?', '!', ',', '.']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))
print(all_words)
print(tags)





["'s", 'a', 'accept', 'anyon', 'are', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'deliveri', 'do', 'doe', 'funni', 'get', 'good', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'my', 'of', 'onli', 'pay', 'paypal', 'see', 'sell', 'ship', 'someth', 'take', 'tell', 'thank', 'that', 'there', 'what', 'when', 'which', 'with', 'you']
['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']


In [17]:
X_train = []
y_train = []

for (pattern_sentence, tag) in xy:
  bag = bag_of_words(pattern_sentence, all_words)
  X_train.append(bag)

  label = tags.index(tag)
  y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)




In [18]:
len(all_words)

54

In [19]:
class ChatDataset(Dataset):
  def __init__(self):
    self.n_samples = len(X_train)
    self.x_data = X_train
    self.y_data = y_train

  def __getitem__(self, index):
    return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.n_samples


In [31]:
dataset

In [27]:
batch_size = 8
hidden_size = 8
output_size = len(tags)
input_size = len(X_train[0])
learning_rate = 0.001
num_epochs = 1000


dataset = ChatDataset()
train_loader = DataLoader(dataset = dataset, batch_size=batch_size, shuffle=True, num_workers = 2)

TypeError: ignored

In [21]:
class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(NeuralNet, self).__init__()
    self.l1 = nn.Linear(input_size, hidden_size)
    self.l2 = nn.Linear(hidden_size, hidden_size)
    self.l3 = nn.Linear(hidden_size, num_classes)
    self.relu = nn.ReLU()

  def forward(self, x):
    out = self.relu(self.l1(x))
    out = self.relu(self.l2(out))
    return self.l3(out)
    

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [34]:
for epoch in range(num_epochs):
  for (words, labels) in train_loader:
    words = words.to(device)
    labels = labels.to(device)

    #forwards
    outputs = model(words)
    loss = criterion(outputs, labels)

    #backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if (epoch + 1) % 100 == 0:
    print(f'epoch {epoch + 1}/{num_epochs}, loss={loss.item():.4f}')


epoch 100/1000, loss=1.1906
epoch 200/1000, loss=0.5552
epoch 300/1000, loss=0.1010
epoch 400/1000, loss=0.0090
epoch 500/1000, loss=0.0134
epoch 600/1000, loss=0.0120
epoch 700/1000, loss=0.0017
epoch 800/1000, loss=0.0023
epoch 900/1000, loss=0.0030
epoch 1000/1000, loss=0.0014


In [29]:
labels.size()

torch.Size([8])

In [30]:
outputs.size()

torch.Size([8, 7])

In [37]:
data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "output_size": output_size,
    "hidden_size": hidden_size,
    "all_words": all_words,
    "tags" : tags
}


FILE = 'data.pth'
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')


training complete. file saved to data.pth


In [39]:
input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model.load_state_dict(model_state)
model.eval()

bot_name = "Sam"
print("Let's chat! (type 'quit' to exit)")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")

Let's chat! (type 'quit' to exit)
You: hello
Sam: Hey :-)
You: what do you sell?
Sam: We sell coffee and tea
You: how much is coffee
Sam: Hi there, how can I help?
You: quit


In [ ]:
quit